In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# EDA

In [ ]:
data=pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
print(data.shape)
data

In [ ]:
data.isnull().sum()
data.dropna(subset=['keyword'], inplace=True)
data.isnull().sum()

In [ ]:
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(strategy='most_frequent')
data['location']=imputer.fit_transform(data[['location']])
data.isnull().sum()

In [ ]:
data

In [ ]:
X=data.iloc[:,1:-1]
Y=data.iloc[:,-1]
print(X.shape,Y.shape)

# Data Cleaning

In [ ]:
X.iloc[:10,2]
for i in range(10):
    print(X.iloc[i,2])
    print()

#remove tags
#remove links
#remove stopwords
#remove emojis

In [ ]:
from nltk.corpus import stopwords
from string import punctuation
stop=stopwords.words('english')
stop.remove('not')

def deEmojify(inputString):
    return inputString.encode('ascii','ignore').decode('ascii')
def hasNumber(string):
    return any(char.isdigit() for char in string)

In [ ]:
import re
clean_text=[]
for tweet,keyword,location in X.loc[:,["text","keyword","location"]].values:
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',tweet) #remove links
    tweet = re.sub('@[^\s]+','',tweet) #remove usernames
    tweet = re.sub('[\s]+', ' ', tweet) #remove additional whitespaces
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) #replace #word with word
    tweet = tweet.strip('\'"') #trim tweet  
    
    words=""
    for word in tweet.split():
        if(word.lower() not in stop):
            word=deEmojify(word)
            words+=(word.lower())+" "
    words+keyword+" "+location
    clean_text.append(words)
print(clean_text[:10])
X['text']=clean_text

In [ ]:
X

# Prepareing Vocabulary

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
vector=TfidfVectorizer(analyzer='word',max_features=5000,max_df=0.8)
#vector = CountVectorizer(max_features=3000)
X=vector.fit_transform(X["text"])


In [ ]:
# from sklearn.svm import SVC
# clf=SVC(kernel='linear')
# clf.fit(X,Y)
# clf

# Model Training

In [ ]:
ann=tf.keras.Sequential()
ann.add(tf.keras.layers.Dense(units=10,activation='relu'))
ann.add(tf.keras.layers.Dense(units=10,activation='relu'))
ann.add(tf.keras.layers.Dense(units=10,activation='relu'))
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


In [ ]:
X_dense = X.toarray() 
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_dense=sc.fit_transform(X_dense)

ann.fit(X_dense,Y,batch_size=32,epochs=100)

# Predictions

In [ ]:
test=data=pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
test

In [ ]:
print(test.isnull().sum())
test['keyword'].fillna("", inplace=True)
test['location']=imputer.transform(test[['location']])
test.isnull().sum()

In [ ]:
test.shape

In [ ]:
test

In [ ]:
import re
clean_text=[]
for tweet,keyword,location in test.loc[:,["text","keyword","location"]].values:
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',tweet) #remove links
    tweet = re.sub('@[^\s]+','',tweet) #remove usernames
    tweet = re.sub('[\s]+', ' ', tweet) #remove additional whitespaces
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) #replace #word with word
    tweet = tweet.strip('\'"') #trim tweet  
    
    words=""
    for word in tweet.split():
        if(word.lower() not in stop):
            word=deEmojify(word)
            words+=(word.lower())+" "
    words+keyword+" "+location
    clean_text.append(words)
print(clean_text[:10])
test['text']=clean_text
id=test['id']

In [ ]:
test=vector.transform(test["text"])
test_dense=test.toarray()

test_dense=sc.transform(test_dense)

In [ ]:
#predictions=clf.predict(test)
predictions = ann.predict(test_dense)
predictions=predictions>0.5
predictions = predictions.astype(int)

In [ ]:
submission_df = pd.DataFrame({'id': id, 'target': predictions.flatten()})
submission_df.to_csv('/kaggle/working/output.csv',index=False)
print("saved")

In [ ]:
review="so give me reasons to fill the hole and merge the space between , let the floods cross distance in your eyes"
vectorized=vector.transform([review])
vectorized=vectorized.toarray()
predicted=ann.predict(vectorized)
print(predicted>0.5)